### GIAI ĐOẠN 1: LẤY DỮ LIỆU TRÊN WEBSITE
---
- **Về công cụ**: Sử dụng Selenimum và BeautifulSoup để lấy dữ liệu tự động từ website.
- **Điều kiện tiên quyết**: Có cài đặt Chrome drivers và thư viện Selenium.
---

#### **1. Import các thư viện cần thiết cho crawling data:**

In [42]:
# Thư viện beautifulSoup:
from bs4 import BeautifulSoup

# Thư viện selenium và lấy ra webdriver:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Thư viện time để sử dụng các hàm liên quan đến thời gian
import time

# Thư viện xử lý mảng, dataframe: 
import pandas as pd
import numpy as np

#### **2. Tiến hành các bước lấy dữ liệu:**

In [33]:
# Mở trình duyệt Chrome ở chế độ toàn màn hình:
browser = webdriver.Chrome()
browser.maximize_window()

In [34]:
# Mở trang chủ của trang cần lấy dữ liệu, ở đây là website Premierleague.com
browser.get('https://www.premierleague.com/players')

---

##### *Giai đoạn 1: Tìm hiểu website và xử lý lần vào đầu tiên*
- **Một số lưu ý** khi mới bắt đầu vào website:
  - Khi mở website trên lên, mới bắt đầu vào ta sẽ thấy một bảng thông báo là phải **Accept all cookies** thì ta mới <br> có thể thực hiện tiếp trên dữ liệu.
  - Có một điểm bất cập ở đây là khi mới bắt đầu vào trang, ta phải đợi nó tải hết một số dữ liệu nên sẽ bị trễ một khoảng thời gian. <br>Với thời gian thực thi của cell tìm và click vào button accept all cookies, nó luôn luôn báo lỗi nên chúng em phải dùng phương thức <br> **time.sleep(3)** để hoãn lại 3 giây chờ web tải hoàn chỉnh rồi sau đó mới có thể nhấn vào được nút.
  - 3 giây là thời gian chúng em quan sát được để website tải được hoàn chỉnh ở môi trường lý tưởng (băng thông mạnh,...).
- Lỗi trả về và cách sửa lỗi được tham khảo: [Stackoverflow](https://stackoverflow.com/questions/65120003/elementnotinteractableexception-using-selenium-in-python)
  
---

In [35]:
# Phương thức delay thời gian tải trang.
time.sleep(5)   

# button_Class: Kiểm tra trên website rồi lấy thủ công về.
button_Class = '//button[@class="_2hTJ5th4dIYlveipSEMYHH BfdVlAo_cgSVjDUegen0F js-accept-all-close"]'

# Tìm chỗ button_class đó rồi thực hiện click vào (Accept all cookies).
browser.find_element(By.XPATH, button_Class).click()

----
- **Kiểu tải dữ liệu của trang:**
  - Giống như Facebook hay Twitter, trang này sẽ tải gần như vô hạn khi tiếp tục kéo xuống dưới cuối.
  - Nhóm em thực hiện đoạn code liên tục kéo xuống trang dữ liệu cho đến khi nào nó tải mới, làm hoài như vậy cho đến khi trang đưa ra những dữ liệu cuối cùng.
- Kéo trang để lấy hết đoạn html đầy đủ chứa các tag href - chứa toàn bộ link các cầu thủ.
- Đoạn code kéo trang được tham khảo tại: [Python forum](https://python-forum.io/thread-20175.html)
----

In [36]:
# Thời gian đợi cuộn tải đữ liệu
SCROLL_PAUSE_TIME = 20

# Lấy kích thước trang ở cuối.
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Cuộn đến cuối trang.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Cài đặt thời gian để load trang khi nó mở rộng thêm dữ liệu.
    time.sleep(SCROLL_PAUSE_TIME)

    # Tính toán với chiều dài cuộn mới và so sánh với chiều cao trang ban đầu.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

---

##### *Giai đoạn 2: Lấy từng chỉ số cầu thủ và đưa ra kết quả cuối cùng.*

- Sử dụng song song các thư viện Selenium và thư viện BeautifulSoup để thực hiện việc lấy dữ liệu website chứa thông tin từng cầu thủ.
- Vì là dạng tổ chức theo các trang nên việc lấy dữ liệu sẽ rất lâu (chạy 146 phút mới hoàn thành việc lấy hết dữ liệu).
---

In [37]:
# List chứa danh sách các liên kết đến thông tin cá nhân của một cấu thủ.
href_pos_lst = []

# Lấy toàn bộ source html trên trang. 
html_text = browser.page_source
soup = BeautifulSoup(html_text)

# Đóng trang web: 
browser.close()

In [45]:
# Lấy các thẻ chứa vị trí của các cầu thủ
player_pos = soup.find_all('td', 'hide-s')
player_pos = player_pos[0::2]

for i in range(len(player_pos)):
    player_pos[i] = player_pos[i].get_text()

player_pos = np.array(player_pos)

In [60]:
# Lấy các thẻ chứa các đường dẫn của các cầu thủ
player_href = []
player_link = soup.find_all('a', 'playerName')

# Xử lý link để truy cập vào link nhanh
for link in player_link: 
    raw_link = 'https:' + link.get('href')
    true_link = raw_link.replace('overview', 'stats')
    player_href.append(true_link)

player_href = np.array(player_href)

In [75]:
# Biến 2 mảng thông tin trên thành mảng 2 chiều để dễ xử lý
href_pos_lst = np.column_stack((player_pos, player_href))

In [122]:
# Làm việc trên danh sách cầu thủ: 
forward_player_data = [] 
mid_player_data = [] 
def_player_data = [] 
keeper_player_data = [] 

# Mở lại trình duyệt và tiến hành vào từng cầu thủ
browser = webdriver.Chrome()
browser.maximize_window()

# Vòng lặp duyệt từng link href để vào đượt trang thống kê chỉ số từng cầu thủ.
for index, link in enumerate(href_pos_lst):
    browser.get(link[1])

    # Lúc mới đầu vào sẽ có nút chấp nhận cookies, xử lý nó.
    if (index == 0):
        time.sleep(4)
        button_Class = '//button[@class="_2hTJ5th4dIYlveipSEMYHH BfdVlAo_cgSVjDUegen0F js-accept-all-close"]'
        browser.find_element(By.XPATH, button_Class).click()
        
    html_text = browser.page_source
    soup = BeautifulSoup(html_text)
    
    # Tìm ra tất cả các thẻ như tên, các mục thống kê rồi lấy các nội dung từ đó.
    name = soup.find('div', 'name t-colour').get_text()
    topStat = soup.find_all('span', 'allStatContainer')

    # Đưa các nội dung vào mảng một chiều như một phần tử của mảng 2 chiều toàn bộ tập dữ liệu.
    one_player = [st.get_text().strip() for st in topStat]
    one_player.insert(0, name)
    
    if (link[0] == 'Goalkeeper'):
        keeper_player_data.append(one_player)
    elif (link[0] == 'Forward'):    
        forward_player_data.append(one_player)
    elif (link[0] == 'Midfielder'):   
        mid_player_data.append(one_player)     
    elif (link[0] == 'Defender'):   
        def_player_data.append(one_player)  

browser.close() 

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=104.0.5112.101)
Stacktrace:
#0 0x561739ff5403 <unknown>
#1 0x561739dfb778 <unknown>
#2 0x561739de9ae5 <unknown>
#3 0x561739de9545 <unknown>
#4 0x561739de8995 <unknown>
#5 0x561739de765b <unknown>
#6 0x561739de7c1c <unknown>
#7 0x561739df51e5 <unknown>
#8 0x561739de9296 <unknown>
#9 0x561739dea232 <unknown>
#10 0x561739de9562 <unknown>
#11 0x561739de8995 <unknown>
#12 0x561739de765b <unknown>
#13 0x561739de7ac2 <unknown>
#14 0x561739dfd1e2 <unknown>
#15 0x561739e643cf <unknown>
#16 0x561739e50a02 <unknown>
#17 0x561739e63c74 <unknown>
#18 0x561739e508f3 <unknown>
#19 0x561739e260d8 <unknown>
#20 0x561739e27205 <unknown>
#21 0x56173a03ce3d <unknown>
#22 0x56173a03fdb6 <unknown>
#23 0x56173a02613e <unknown>
#24 0x56173a0409b5 <unknown>
#25 0x56173a01a970 <unknown>
#26 0x56173a05d228 <unknown>
#27 0x56173a05d3bf <unknown>
#28 0x56173a077ab

In [ ]:
dfForward = pd.DataFrame(forward_player_data)
dfForward.columns = ['Player name', 'Appearances', 'Goals Overview', 'Wins', 'Losses', 'Goals', 'Goals per match',
                    'Headed goals', 'Goals with right foot', 'Goals with left foot',
                    'Penalties scored', 'Freekicks scored', 'Shots',
                    'Shots on target', 'Shooting accuracy', 'Hit woodwork',
                    'Big chances missed', 'Assists', 'Passes', 'Passes per match',
                    'Big Chances Created', 'Crosses', 'Yellow cards', 'Red cards',
                    'Fouls', 'Offsides', 'Tackles', 'Blocked shots',
                    'Interceptions', 'Clearances', 'Headed Clearance']

In [ ]:
dfForward.to_csv('forward.csv', index=False)

In [ ]:
dfMid = pd.DataFrame(mid_player_data)
dfMid.columns = ['Player name', 'Appearances', 'Goals Overview', 'Wins', 'Losses', 'Goals', 'Goals per match',
                    'Headed goals', 'Goals with right foot', 'Goals with left foot',
                    'Penalties scored', 'Freekicks scored', 'Shots', 'Shots on target', 
                    'Shooting accuracy %', 'Hit woodwork', 'Big chances missed', 'Assists',
                    'Passes', 'Passes per match', 'Big chances created', 'Crosses', 'Cross accuracy %',
                    'Through balls', 'Accurate long balls', 'Yellow cards', 'Red cards',
                    'Fouls', 'Offsides', 'Tackles', 'Tackle success %', 'Blocked shots',
                    'Interceptions', 'Clearances', 'Headed clearances', 'Recoveries', 'Duels won',
                    'Duels lost', 'Succesful 50/50s', 'Aerial battles won', 'Aerial battles lost', 
                    'Errors leading to goal']


In [ ]:
dfMid.to_csv('midfielder_data.csv', index=False)

In [107]:
dfDef = pd.DataFrame(def_player_data)
dfDef.columns = ['Player name', 'Appearances', 'Goals Overview', 'Wins', 'Losses', 'Clean sheets', 
                'Goals Conceded', 'Tackles', 'Tackle success' , 'Last man tackles',
                'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance',
                'Clearances off line', 'Recoveries', 'Duels won', 'Duels lost', 'Successful 50/50s',
                'Aerial battles won', 'Aerial battles lost', 'Own goals', 'Errors leading to goal',
                'Assists', 'Passes', 'Passes per match', 'Big Chances Created', 'Crosses',
                'Cross accuracy' , 'Through balls', 'Accurate long balls', 'Yellow cards',
                'Red cards', 'Fouls', 'Offsides', 'Goals', 'Headed goals', 'Goals with right foot',
                'Goals with left foot', 'Hit woodwork']

39

In [ ]:
dfDef.to_csv('defender_data.csv', index=False)

In [115]:
dfKeeper = pd.DataFrame(keeper_player_data)
dfKeeper.columns = ['Player name', 'Appearances', 'Goals', 'Wins', 'Losses', 'Saves',
            'Penalties Saved', 'Punches', 'High Claims','Catches',
            'Sweeper clearances', 'Throw outs', 'Goal Kicks',
            'Clean sheets', 'Goals Conceded', 'Errors leading to goal',
            'Own goals', 'Yellow cards', 'Red cards', 'Fouls', 'Goals',
            'Assists', 'Passes', 'Passes per match', 'Accurate long balls']

In [117]:
dfKeeper.to_csv('keeper_data.csv', index=False)

#### **3. Tiến hành đưa dữ liệu về file csv chung:**
- Sử dụng thư viện Pandas để đưa mảng chứa các dữ liệu cầu thủ về loại DataFrames của Pandas.
- Sau khi chuyển về xong, ta viết các thông tin của Dataframe đó về một file CSV và đánh các tên chỉ số cho các cột dựa trên các thuộc tính lấy được.
- Viết lại vào một file csv để tiện dùng cho sau này.

---
#### 4. Đánh giá và báo cáo giai đoạn crawling data:
- Đánh giá bước lấy dữ liệu: 
  - **Hạn chế**: 
    - Việc thực thi các cell lấy dữ liệu mất nhiều thời gian (146 phút) để có thể lấy được toàn bộ dữ liệu. Nguyên nhân một phần là vì các thao tác như sang trang cầu thủ, bấm vào nút thống kê để xem chi tiết, lấy các dữ liệu cần thiết... Và cũng một phần là phải thực hiện kéo trang đến cuối để có thể hiện được hết thông tin.
    - Thay đổi trang web vì phải tìm được trang ít lỗi và tĩnh nhất có thể.
    - Có một vài trang dữ liệu có định dạng khác format thông thường vì các cầu thủ đó chưa đá nên chưa có dữ liệu, bắt buộc ở bước tiền xử lý dữ liệu thực hiện sau.
    - Giai đoạn hoàn chỉnnh mất nhiều tuần.
  - **Thuận lợi:**
    - Không.
  